# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Задача данного проекта: исследование зависимости факта погашения кредита в срок от семейного статуса и наличия детей. В работе будет использована библиотека pandas, pymystem3 и контейнер Counter из модуля collections. Основной акцент в данной работе будет сделан на предобработке данных, потому что, во-первых, проект учебный, а во-вторых, предобработка данных - самый трудоёмкий этап в анализе данных. Ошибки на этом этапе могут стоить дорого как для исполнителя проекта, так и для инвестора, заказчика или любого другого бенефициара.


## Откроем файл с данными и изучим общую информацию

In [1]:
import pandas
data = pandas.read_csv('/datasets/data.csv')
display(data.head(10))
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В таблице 12 колонок с тремя типами данных: float64, int64, object. 

- Странным выглядит столбец days_employed, в нём отрицательные значения; также есть пропуски (в нём всего 19351 значений вместо 21525). 

- В столбце education нужно привести значения к нижнему регистру. 

- Столбец total_income также пострадал от пропусков значений. Полагаю, что надо будет категоризировать заёмщиков и по каждой категории посчитать медиану и заполнить ими пропуски, чтобы датафрейм стал релевантным для анализа.

- Столбец purpose с житейской точки зрения также интересен для анализа, потому что брать кредить на свадьбу - это очень интересно. Наверняка у таких людей интересная кредитная история и вообще жизнь (это не относится к задаче, но я бы на месте заказчика поанализировала эту категорию заёмщиков в ретроспективе). Полагаю, что этот столбец надо категоризировать с помощью лемматизации.

- Столбцы days_employed и total_income имеют одинаковое количество пропусков. Наверняка какая-то техническая ошибка.

## Предобработка данных

### Обработка пропусков

In [2]:
display('до обработки количество пропусков в days_employed:', data['days_employed'].isna().sum())
medians_de = (data.groupby(['family_status', 'education']).agg({'days_employed': 'median'}).rename(columns = {'days_employed': 'median_days_employed'}))
data = data.merge(medians_de, on = ['family_status', 'education'])
data.loc[data['days_employed'].isna(), 'days_employed'] = data.loc[data['days_employed'].isna(), 'median_days_employed']
display('после обработки количество пропусков:', data['days_employed'].isna().sum())
#data.info()
#display(data.head())

display('до обработки количество пропусков в total_income:', data['total_income'].isna().sum())
medians_ti = (data.groupby(['income_type']).agg({'total_income': 'median'}).rename(columns = {'total_income': 'median_total_income'}))
data = data.merge(medians_ti, on = ['income_type'])
data.loc[data['total_income'].isna(), 'total_income'] = data.loc[data['total_income'].isna(), 'median_total_income']
display('после обработки количество пропусков:',data['total_income'].isna().sum())
data.info()

'до обработки количество пропусков в days_employed:'

2174

'после обработки количество пропусков:'

0

'до обработки количество пропусков в total_income:'

2174

'после обработки количество пропусков:'

0

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children                21525 non-null int64
days_employed           21525 non-null float64
dob_years               21525 non-null int64
education               21525 non-null object
education_id            21525 non-null int64
family_status           21525 non-null object
family_status_id        21525 non-null int64
gender                  21525 non-null object
income_type             21525 non-null object
debt                    21525 non-null int64
total_income            21525 non-null float64
purpose                 21525 non-null object
median_days_employed    21525 non-null float64
median_total_income     21525 non-null float64
dtypes: float64(4), int64(5), object(5)
memory usage: 2.5+ MB


**Вывод**

**Столбец days_employed**
1. В столбце было 2174 пропусков (равно как и в столбце total_income, поэтому предполагаю техническую ошибку).
2. В столбце тип данных - вещественное число, да ещё и есть отрицательные значения.
3. Пропуски были заполнены медианными значениями, сгруппированными по столбцам family_status, education.

**Столбец total_income**
1. В столбце 2174 пропусков.
2. Тип данных - вещественные числа.
3. Группировку и медиану сделала по столбцу income_type, предположив, что доход зависим от его типа.

### Замена типа данных

In [3]:
data['days_employed'] = data['days_employed'].apply(abs)
data.loc[data['days_employed'] > 29000, 'days_employed'] = data.loc[data['days_employed'] >29000, 'days_employed'] / 24
data['days_employed'] = data['days_employed'].astype(int)
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children                21525 non-null int64
days_employed           21525 non-null int64
dob_years               21525 non-null int64
education               21525 non-null object
education_id            21525 non-null int64
family_status           21525 non-null object
family_status_id        21525 non-null int64
gender                  21525 non-null object
income_type             21525 non-null object
debt                    21525 non-null int64
total_income            21525 non-null float64
purpose                 21525 non-null object
median_days_employed    21525 non-null float64
median_total_income     21525 non-null float64
dtypes: float64(3), int64(6), object(5)
memory usage: 2.5+ MB


**Вывод**

Трудовой стаж в днях имеет тип "вещественное число". Нужно заменить на положительное, целое и привести к единой системе измерения, хотя этот столбец не входит в задачу по выяснению зависимости факта погашения кредита от семейного положения и количества детей клиента (просто увидела в обсуждениях, что без обработки этого столбца ревьюер не примет проект)))

Я приняла 29000 как максимальный стаж в днях для человека (80 лет).

### Обработка дубликатов

In [4]:
display('в таблице явных дубликатов:', data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)
#display(data.duplicated().sum())
display(data['family_status'].unique())
display(data['education'].unique())
display(data['children'].unique())
data['education'] = data['education'].str.lower()
display('Уникальные значения в столбце education должны выгляеть так:', data['education'].unique())

'в таблице явных дубликатов:'

54

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень', 'Ученая степень'],
      dtype=object)

array([ 1,  0,  2,  3,  4, -1, 20,  5])

'Уникальные значения в столбце education должны выгляеть так:'

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Вывод**

1. Удаляем явные дубликаты методом drop_duplicates и обновляем идексы. 
2. В столбце family_status нет дубликатов; в столбце children тоже нет (но значение -1 странное); а вот в столбце education есть: желательно было привести все значения в нижний регистр и проверить результат.
3. Причины появления дубликатов: человеческий фактор - источник ошибок)). Также возможны нюансы в выгрузке данных или ошибки при их обогащении.

### Лемматизация

In [5]:
#display(data['purpose'].unique())
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
lemmas = m.lemmatize(' '.join(data['purpose']))
lemmas = Counter(lemmas)
lemmas.most_common(15)

[(' ', 55066),
 ('недвижимость', 6353),
 ('покупка', 5900),
 ('жилье', 4461),
 ('автомобиль', 4308),
 ('образование', 4014),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2335),
 ('свой', 2231),
 ('на', 2228),
 ('строительство', 1879),
 ('высокий', 1374),
 ('получение', 1315),
 ('коммерческий', 1312)]

**Вывод**

Сначала мне было интересно посмотреть все уникальные значения в столбце purpose. Сразу стало ясно, что необходима лемматизация и подсчёт самых упоминаемых лемм, чтобы в дальнейшем проводить категоризацию.

Я импортровала контейнер Counter из модуля collections, создала список лемм, посчитала их и посмотрела топ-15 самых употребляемых.

Полагаю, что категоризировать нужно так:
- если леммы: недвижимость, жильё, строительство то отнесём это к категории цели "операции с недвижимостью"
- если леммы: автомобиль - "покупка автомобиля"
- свадьба - "организация свадьбы"
- образование - "получение образования"
- иначе - "прочее"

### Категоризация данных

In [6]:
def categorize(purpose):
    purpose_lemmas = m.lemmatize(purpose)
    if 'недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas or 'строительство' in purpose_lemmas:
        return 'операции с недвижимостью'
    if 'автомобиль' in purpose_lemmas:
        return 'покупка автомобиля'
    if 'образование' in purpose_lemmas:
        return 'получение образования'
    if 'свадьба' in purpose_lemmas:
        return 'организация свадьбы'
    return 'прочее'

data['purpose_category'] = data['purpose'].apply(categorize)

display(data[['purpose', 'purpose_category']].head(20))


,purpose,purpose_category
0,покупка жилья,операции с недвижимостью
1,строительство собственной недвижимости,операции с недвижимостью
2,строительство собственной недвижимости,операции с недвижимостью
3,недвижимость,операции с недвижимостью
4,на покупку подержанного автомобиля,покупка автомобиля
5,жилье,операции с недвижимостью
6,операции с жильем,операции с недвижимостью
7,автомобиль,покупка автомобиля
8,профильное образование,получение образования
9,недвижимость,операции с недвижимостью


**Вывод**

Категоризация была проведена в соответствии с выводами в предыдущем пункте. После категоризации я присоединила столбец purpose_category к таблице.

In [7]:
display(data[['education_id', 'education']].drop_duplicates())
display(data[['family_status', 'family_status_id']].drop_duplicates())
data['family_status'] = data['family_status'].str.lower()
display(data[['family_status', 'family_status_id']].drop_duplicates())

,education_id,education
0,0,высшее
1249,1,среднее
8007,2,неоконченное высшее
9340,3,начальное
11081,4,ученая степень


,family_status,family_status_id
0,женат / замужем,0
5809,гражданский брак,1
8159,вдовец / вдова,2
8377,в разводе,3
8400,Не женат / не замужем,4


,family_status,family_status_id
0,женат / замужем,0
5809,гражданский брак,1
8159,вдовец / вдова,2
8377,в разводе,3
8400,не женат / не замужем,4


## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
children_correlation = data.groupby('children').agg({'debt': ['count', 'sum', 'mean']})
children_correlation.columns = ['total', 'debt', '%']
display(children_correlation)

,total,debt,%
children,,,
-1,47,1,0.021277
0,14107,1063,0.075353
1,4809,444,0.092327
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000
20,76,8,0.105263


**Вывод**

Да, те, у кого есть дети, более склонны к образованию просроченной кредиторской задолженности. Берут кредиты чаще бездетные в этом датасете.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
family_status_correlation = data.groupby('family_status').agg({'debt': ['count', 'sum', 'mean']})
family_status_correlation.columns = ['total', 'debt', '%']
display(family_status_correlation)

,total,debt,%
family_status,,,
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693
гражданский брак,4163,388,0.093202
женат / замужем,12344,931,0.075421
не женат / не замужем,2810,274,0.097509


**Вывод**

Холостые, незамужние и состоящие в гражданском браке реже возвращают кредит в срок. А женатые, замужние и разведённые лучше исполняют условия кредитного договора. А вот мечта банков, как это ни цинично звучит - овдовевшие люди, но они и реже всех берут кредит.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
def categorize_income(total_income):
    median_total_income = data['total_income'].median()

    if total_income <= 0.5 * median_total_income:
        return 'низкий уровень дохода'
    if total_income >= 1.5 * median_total_income:
        return 'высокий уровень дохода'
    return 'средний уровень дохода'

data['income_category'] = data['total_income'].apply(categorize_income)

display(data[['total_income', 'income_category']].head(10))


,total_income,income_category
0,253875.639453,высокий уровень дохода
1,308848.983691,высокий уровень дохода
2,187863.237306,средний уровень дохода
3,414404.202897,высокий уровень дохода
4,148992.030010,средний уровень дохода
5,142594.396847,средний уровень дохода
6,142594.396847,средний уровень дохода
7,94187.878082,средний уровень дохода
8,142594.396847,средний уровень дохода
9,280934.305869,высокий уровень дохода


In [11]:
income_correlation = data.pivot_table(index = 'income_category', columns = 'debt', values = 'total_income', aggfunc = 'count')
income_correlation.columns = ['no_debt', 'debt']
income_correlation['%'] = income_correlation['debt'] / (income_correlation['debt'] + income_correlation['no_debt'])
display(income_correlation)

,no_debt,debt,%
income_category,,,
высокий уровень дохода,4016,304,0.070370
низкий уровень дохода,1470,109,0.069031
средний уровень дохода,14244,1328,0.085281


**Вывод**

Для начала необходимо было категоризировать доход на низкий, средний и высокий. Всё, что ниже 0,5 медианы, я причислила к низкому, всё, что выше 1,5 медианы - к высокому. Остальное - средний доход.

Как это ни парадоксально, люди с низким уровнем дохода более дисциплинированы в кредитных отношениях, чем люди с высоким уровнем дохода. А у "середнячков" наибольшая вероятность уйсти в просрочку.


- Как разные цели кредита влияют на его возврат в срок?

In [12]:
purpose_category_correlation = data.groupby('purpose_category').agg({'debt': ['count', 'sum', 'mean']})
purpose_category_correlation.columns = ['total', 'debt', '%']
display(purpose_category_correlation)

,total,debt,%
purpose_category,,,
операции с недвижимостью,10814,782,0.072314
организация свадьбы,2335,186,0.079657
покупка автомобиля,4308,403,0.093547
получение образования,4014,370,0.092177


**Вывод**

Автомобилисты и студенты - наименее надёжные кредиторы. А берущие кредит на свадьбу имеют ровно такой же риск просрочки как и домовладельцы.

## Общий вывод

Датасет, предложенный к анализу, содержал в себе много информации, местами сырой, которую нужно было "довести до ума", чтобы сделать пригодной к анализу. 
Сначала был проведён общий обзор таблицы, который показал общую картину, ознакомил с названиями столбцов и данными внутри них. Также общий обзор помогает построить дизайн исследования, определить его этапы, чтобы решить поставленную задачу.
Данных для подтверждения или опровержения гипотез достаточно - в таблице 21525 строк и 12 столбцов.

В столбце days_employed имеются отрицательные значения, столбец children имеет значение -1, столбцы days_employed и total_income имеют одинаковое количество пропусков. Возможно, это техническая ошибка, с которой следует поразбираться.
Со столбцами days_employed и total_income была проведена работа по нахождению пропусков, замене значений. В	days_employed пропуски были заполнены медианными значениями, сгруппированными по столбцам family_status, education. В total_income группировка и медиана были сделаны по столбцу income_type, предположив, что доход зависим от его типа.

Трудовой стаж в днях имел тип "вещественное число", был заменён на положительное, целое и данные приведены к единой системе измерения.

Явные дубликаты были удалены методом drop_duplicates с обновлением идексов.

В столбце education все значения были приведены в нижний регистр.

Есть предположение, что пропуски, дубликаты, отрицательные значения связаны с выгрузкой данных или/и ошибках при их обогащении.

С помощью лемматизации была произведена категоризация данных:
-	если леммы: недвижимость, жильё, строительство, то были отнесены к категории цели "операции с недвижимостью"
-	если леммы: автомобиль - "покупка автомобиля"
-	свадьба - "организация свадьбы"
-	образование - "получение образования"
-	иначе - "прочее"

Выбор лемм был сделан по частоте встречаемости в таблице. После категоризации я присоединила столбец purpose_category к таблице. 

После работы по предобработке данных стало возможным выяснить корреляцию. Были использованы groupby, agg и pivot_table.

Те, у кого есть дети, более склонны к образованию просроченной кредиторской задолженности. Берут кредиты чаще бездетные в этом датасете.
Холостые, незамужние и состоящие в гражданском браке реже возвращают кредит в срок. А женатые, замужние и разведённые лучше исполняют условия кредитного договора. А вот мечта банков, как это ни цинично звучит - овдовевшие люди, но они и реже всех берут кредит.
Чтобы понять зависимость от дохода, нужно было сначала его категоризировать на низкий, средний и высокий. Всё, что ниже 0,5 медианы, было причислено к низкому, всё, что выше 1,5 медианы - к высокому. Остальное - средний доход.
Как это ни парадоксально, люди с низким уровнем дохода более дисциплинированы в кредитных отношениях, чем люди с высоким уровнем дохода. А у "середнячков" наибольшая вероятность уйсти в просрочку.
По целям кредита результат следующий: автомобилисты и студенты - наименее надёжные кредиторы. А берущие кредит на свадьбу имеют  такой же риск просрочки как и домовладельцы.

Итак, поставленные задачи выполнены, однако данные говорят о том, что они ещё не окончательно проанализированы. Для разработки модели кредитного скоринга необходимо выяснить и другие корреляции. Например, между возвратом долга в срок и трудовым стажем или полом.